In [ ]:
!pip install easyocr
!pip install gradio


!pip install transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [12]:
import easyocr
import cv2
import numpy as np
import gradio as gr
from transformers import pipeline

# Define OCR functions
def ocr_core_1(Img_path):
    reader = easyocr.Reader(["ja"], gpu=True)  # lang 1 Japanese
    result = reader.readtext(Img_path)
    avg = 0
    result_txt = ""
    for i in result:
        avg += i[2]
        result_txt += i[1]
    avg /= len(result)
    return avg, result_txt

def ocr_core_2(Img_path):
    reader = easyocr.Reader(['ko'], gpu=True)  # lang 2 Korean
    result = reader.readtext(Img_path)
    avg = 0
    result_txt = ""
    for i in result:
        avg += i[2]
        result_txt += i[1]
    avg /= len(result)
    return avg, result_txt

def ocr_core_3(Img_path):
    reader = easyocr.Reader(['hi'], gpu=True)  # lang 3 Hindi
    result = reader.readtext(Img_path)
    avg = 0
    result_txt = ""
    for i in result:
        avg += i[2]
        result_txt += i[1]
    avg /= len(result)
    return avg, result_txt

def ocr_core_4(Img_path):
    reader = easyocr.Reader(['te'], gpu=True)  # lang 4 Telugu
    result = reader.readtext(Img_path)
    avg = 0
    result_txt = ""
    for i in result:
        avg += i[2]
        result_txt += i[1]
    avg /= len(result)
    return avg, result_txt

def ocr_core_5(Img_path):
    reader = easyocr.Reader(['ar'], gpu=True)  # lang 5 Arabic
    result = reader.readtext(Img_path)
    avg = 0
    result_txt = ""
    for i in result:
        avg += i[2]
        result_txt += i[1]
    avg /= len(result)
    return avg, result_txt

def ocr_core_6(Img_path):
   reader = easyocr.Reader(['en'], gpu=True)  # lang 6 English
   result = reader.readtext(Img_path)
   avg = 0
   result_txt = ""
   for i in result:
       avg += i[2]
       result_txt += i[1]
   avg /= len(result)
   return avg, result_txt

# Function to handle OCR, language detection, and translation
def process_image(img):
    # Save image to file
    img_path = 'temp_image.png'
    cv2.imwrite(img_path, img)

    # Get OCR results for each language
    score_label = [ocr_core_1(img_path), ocr_core_2(img_path), ocr_core_3(img_path), ocr_core_4(img_path), ocr_core_5(img_path),ocr_core_6(img_path)]

    # Find the language with the highest score
    max_index = score_label.index(max(score_label))
    languages = ["Japanese", "Korean", "Hindi", "Telugu", "Arabic","English"]
    detected_language = languages[max_index]

    extracted_text = score_label[max_index][1]

    # Translation pipeline
    if max_index == 0:
        pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-ja-en")
    elif max_index == 1:
        pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-ko-en")
    elif max_index == 2:
        pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-hi-en")
    elif max_index == 3:
        pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-te-en")
    elif max_index == 4:
        pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-ar-en")
    elif max_index == 5:
        return detected_language, extracted_text, extracted_text

    translated_text = pipe(extracted_text)[0]['translation_text']

    return detected_language, extracted_text, translated_text

# Function to handle feedback submission
def submit_feedback(feedback):
    # Here you can process the feedback, e.g., save to a file or send to a server
    print("Feedback received:", feedback)
    return "Thank you for your feedback!"

# Custom CSS for glass theme and background
custom_css = """
body {
     background-image: url("/content/drive/MyDrive/Screenshot_20221229-172559_Gmail.jpg"); /* Replace with your image path */
    background-size: cover;  /* Adjust how the image covers the background */
    background-repeat: no-repeat;  /* Prevent image repetition */
    background-size: cover;
    background-position: center;
    min-height: 100vh;
    margin: 0;
    font-family: Arial, sans-serif;
}
.gradio-container {
    background-image: url("/content/drive/MyDrive/Screenshot_20221229-172559_Gmail.jpg"); /* Replace with your image path */
    background-size: cover;  /* Adjust how the image covers the background */
    background-repeat: no-repeat;  /* Prevent image repetition */
    border-radius: 20px;
    padding: 20px;
    backdrop-filter: blur(10px);  /* Glass effect */
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
}
.gr-text-input, .gr-textarea, .gr-button {
    border-radius: 10px;
}
"""

# Custom theme and fonts
theme = gr.themes.Default(font=[gr.themes.GoogleFont("Inconsolata"), "Arial", "sans-serif"])


# Gradio interface
with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("<h1 style='text-align: center; color: white;'>OCR and Translation</h1>")
    gr.Markdown("<p style='text-align: center; color: white;'>Upload an image with text in Japanese, Korean, Hindi, Telugu, or Arabic. The app will extract and translate the text to English.</p>")

    with gr.Row():
        image_input = gr.Image(type="numpy", label="Upload Image")

    with gr.Row():
        submit_button = gr.Button("Submit")
        clear_button = gr.Button("Clear")

    with gr.Row():
        detected_language = gr.Textbox(label="Detected Language")
        extracted_text = gr.Textbox(label="Extracted Text")
        translated_text = gr.Textbox(label="Translated Text")

    with gr.Row():
        feedback = gr.Textbox(label="Feedback", placeholder="Leave your feedback here...", lines=2)
        feedback_button = gr.Button("Submit Feedback")

    examples = gr.Examples(
        examples=["Image1.png", "Image2.png", "Image3.png",],
        inputs=image_input,
    )

    def clear():
        return None, None, None, ""

    submit_button.click(fn=process_image, inputs=image_input, outputs=[detected_language, extracted_text, translated_text])
    clear_button.click(fn=clear, outputs=[image_input, detected_language, extracted_text, translated_text])
    feedback_button.click(fn=submit_feedback, inputs=feedback, outputs=feedback)

demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2c0fd6190a4a2cbfeb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
